STEP 1:

DATA CLEANING

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('fakeReviewData.csv')  # Adjust for file type, e.g., .csv or .xlsx

In [ ]:
print(df.head())

             category  rating label  \
0  Home_and_Kitchen_5     5.0    CG   
1  Home_and_Kitchen_5     5.0    CG   
2  Home_and_Kitchen_5     5.0    CG   
3  Home_and_Kitchen_5     1.0    CG   
4  Home_and_Kitchen_5     5.0    CG   

                                               text_  
0  Love this!  Well made, sturdy, and very comfor...  
1  love it, a great upgrade from the original.  I...  
2  This pillow saved my back. I love the look and...  
3  Missing information on how to use it, but it i...  
4  Very nice set. Good quality. We have had the s...  


In [ ]:
#here we can see what the data looks like, the shape, the number of null values, column names

print(df.shape)
print(df.columns)
print(df.isnull().sum())
print(df.head())

(40432, 4)
Index(['category', 'rating', 'label', 'text_'], dtype='object')
category    0
rating      0
label       0
text_       0
dtype: int64
             category  rating label  \
0  Home_and_Kitchen_5     5.0    CG   
1  Home_and_Kitchen_5     5.0    CG   
2  Home_and_Kitchen_5     5.0    CG   
3  Home_and_Kitchen_5     1.0    CG   
4  Home_and_Kitchen_5     5.0    CG   

                                               text_  
0  Love this!  Well made, sturdy, and very comfor...  
1  love it, a great upgrade from the original.  I...  
2  This pillow saved my back. I love the look and...  
3  Missing information on how to use it, but it i...  
4  Very nice set. Good quality. We have had the s...  


In [ ]:
print(df['category'].unique())
print(df['label'].unique())
print(df['rating'].unique())

['Home_and_Kitchen_5' 'Sports_and_Outdoors_5' 'Electronics_5'
 'Movies_and_TV_5' 'Tools_and_Home_Improvement_5' 'Pet_Supplies_5'
 'Kindle_Store_5' 'Books_5' 'Toys_and_Games_5'
 'Clothing_Shoes_and_Jewelry_5']
['CG' 'OR']
[5. 1. 3. 2. 4.]


In [ ]:
#in this dataset, we have every datapoint as a review at an online marketplace.
#here, we can afford to drop duplicates since a duplicate review doesn't hold value for us.

df = df.drop_duplicates()


STEP 2:

TEXT NORMALIZATION

In [ ]:
#here we'll work on the review text itself.
#we want to perform the following:
#1. remove numbers, punctuations, special charactesr
#2. convert all upper case characters to lower case


import re

def clean_text(value):
    value = value.lower()
    value = re.sub(r'[^a-z\s]', '', value)
    return value

df['text_'] = df['text_'].apply(clean_text)

In [ ]:
print(df["text_"].head(15))

0     love this  well made sturdy and very comfortab...
1     love it a great upgrade from the original  ive...
2     this pillow saved my back i love the look and ...
3     missing information on how to use it but it is...
4     very nice set good quality we have had the set...
5           i wanted different flavors but they are not
6     they are the perfect touch for me and the only...
7     these done fit well and look great  i love the...
8     great big numbers  easy to read the only thing...
9     my son loves this comforter and it is very wel...
10    as advertised th one ive had the only problem ...
11    very handy for one of my kids and the tools ar...
12    did someone say oriental for   it is a great p...
13    these are so flimsy they are not the quality y...
14    makes may tea with out stirring the only probl...
Name: text_, dtype: object


STEP 3:

TOKENIZATION


In [ ]:
import spacy

# Load spaCy model

# Apply tokenization
from tqdm import tqdm
tqdm.pandas()

# Tokenize with progress bar
df['tokens'] = df['text_'].progress_apply(lambda x: [token.text for token in nlp(x)])


LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - '/root/nltk_data'
    - '/usr/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


In [ ]:
print(df.head())

             category  rating label  \
0  Home_and_Kitchen_5     5.0    CG   
1  Home_and_Kitchen_5     5.0    CG   
2  Home_and_Kitchen_5     5.0    CG   
3  Home_and_Kitchen_5     1.0    CG   
4  Home_and_Kitchen_5     5.0    CG   

                                               text_  
0  love this  well made sturdy and very comfortab...  
1  love it a great upgrade from the original  ive...  
2  this pillow saved my back i love the look and ...  
3  missing information on how to use it but it is...  
4  very nice set good quality we have had the set...  


the above code snippet took 15 mins to run. i'll now be downloading this dataset to prevent having to run it every time.

In [ ]:
# Export to a CSV file
df.to_csv('fakeReviewDataTokenized.csv', index=False)  # Set index=False to avoid saving the index column


In [ ]:
df = pd.read_csv("fakeReviewDataTokenized.csv")

STEP 4:

STOPWORD REMOVAL


In [ ]:
import ast

# Convert stringified lists to actual lists
df['tokens'] = df['tokens'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# Verify the output
print(df['tokens'].iloc[0])  # Check the first row's tokens
print(type(df['tokens'].iloc[0]))  # Confirm type is a list


['love', 'this', ' ', 'well', 'made', 'sturdy', 'and', 'very', 'comfortable', ' ', 'i', 'love', 'itvery', 'pretty']
<class 'list'>


In [ ]:
from nltk.corpus import stopwords
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))

# Remove stopwords
df['tokens_cleaned'] = df['tokens'].apply(lambda tokens: [word for word in tokens if word.lower() not in stop_words])

# Verify cleaned tokens
print(df['tokens_cleaned'].iloc[0])  # Check the first row's cleaned tokens
print(type(df['tokens_cleaned'].iloc[0]))  # Confirm type is a list


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['love', ' ', 'well', 'made', 'sturdy', 'comfortable', ' ', 'love', 'itvery', 'pretty']
<class 'list'>


In [ ]:
print(df.head())

             category  rating label  \
0  Home_and_Kitchen_5     5.0    CG   
1  Home_and_Kitchen_5     5.0    CG   
2  Home_and_Kitchen_5     5.0    CG   
3  Home_and_Kitchen_5     1.0    CG   
4  Home_and_Kitchen_5     5.0    CG   

                                               text_  \
0  love this  well made sturdy and very comfortab...   
1  love it a great upgrade from the original  ive...   
2  this pillow saved my back i love the look and ...   
3  missing information on how to use it but it is...   
4  very nice set good quality we have had the set...   

                                              tokens  \
0  [love, this,  , well, made, sturdy, and, very,...   
1  [love, it, a, great, upgrade, from, the, origi...   
2  [this, pillow, saved, my, back, i, love, the, ...   
3  [missing, information, on, how, to, use, it, b...   
4  [very, nice, set, good, quality, we, have, had...   

                                      tokens_cleaned  
0  [love,  , well, made, sturdy, com

In [ ]:
# Filter out spaces and empty strings
df['tokens_cleaned'] = df['tokens_cleaned'].apply(lambda tokens: [word for word in tokens if word.strip()])
print(df[['tokens', 'tokens_cleaned']].head())


                                              tokens  \
0  [love, this,  , well, made, sturdy, and, very,...   
1  [love, it, a, great, upgrade, from, the, origi...   
2  [this, pillow, saved, my, back, i, love, the, ...   
3  [missing, information, on, how, to, use, it, b...   
4  [very, nice, set, good, quality, we, have, had...   

                                      tokens_cleaned  
0  [love, well, made, sturdy, comfortable, love, ...  
1  [love, great, upgrade, original, mine, couple,...  
2    [pillow, saved, back, love, look, feel, pillow]  
3  [missing, information, use, great, product, pr...  
4       [nice, set, good, quality, set, two, months]  


STEP 5.1:

STEMMING


In [ ]:
from nltk.stem import PorterStemmer

# Initialize the stemmer
stemmer = PorterStemmer()

# Apply stemming to the cleaned tokens
df['tokens_stemmed'] = df['tokens_cleaned'].apply(lambda tokens: [stemmer.stem(word) for word in tokens])

# Verify the results
print(df[['tokens_cleaned', 'tokens_stemmed']].head())



                                      tokens_cleaned  \
0  [love, well, made, sturdy, comfortable, love, ...   
1  [love, great, upgrade, original, mine, couple,...   
2    [pillow, saved, back, love, look, feel, pillow]   
3  [missing, information, use, great, product, pr...   
4       [nice, set, good, quality, set, two, months]   

                                      tokens_stemmed  
0  [love, well, made, sturdi, comfort, love, itve...  
1   [love, great, upgrad, origin, mine, coupl, year]  
2     [pillow, save, back, love, look, feel, pillow]  
3         [miss, inform, use, great, product, price]  
4        [nice, set, good, qualiti, set, two, month]  


STEP 5.2:

LEMMATIZATION

In [ ]:
import spacy

# Load the spaCy model
nlp = spacy.load("en_core_web_sm")

# Apply lemmatization
df['tokens_lemmatized'] = df['tokens_cleaned'].apply(lambda tokens: [token.lemma_ for token in nlp(" ".join(tokens))])

# Verify results
print(df[['tokens_stemmed', 'tokens_lemmatized']].head())


                                      tokens_cleaned  \
0  [love, well, made, sturdy, comfortable, love, ...   
1  [love, great, upgrade, original, mine, couple,...   
2    [pillow, saved, back, love, look, feel, pillow]   
3  [missing, information, use, great, product, pr...   
4       [nice, set, good, quality, set, two, months]   

                                   tokens_lemmatized  
0  [love, well, make, sturdy, comfortable, love, ...  
1  [love, great, upgrade, original, mine, couple,...  
2     [pillow, save, back, love, look, feel, pillow]  
3    [miss, information, use, great, product, price]  
4        [nice, set, good, quality, set, two, month]  


In [ ]:
print(df[['tokens_stemmed', 'tokens_lemmatized']].head())


                                      tokens_stemmed  \
0  [love, well, made, sturdi, comfort, love, itve...   
1   [love, great, upgrad, origin, mine, coupl, year]   
2     [pillow, save, back, love, look, feel, pillow]   
3         [miss, inform, use, great, product, price]   
4        [nice, set, good, qualiti, set, two, month]   

                                   tokens_lemmatized  
0  [love, well, make, sturdy, comfortable, love, ...  
1  [love, great, upgrade, original, mine, couple,...  
2     [pillow, save, back, love, look, feel, pillow]  
3    [miss, information, use, great, product, price]  
4        [nice, set, good, quality, set, two, month]  


In [ ]:
df.to_csv('fakeReviewDataLemmatized.csv', index=False)  # Set index=False to avoid saving the index column


In [3]:
import pandas as pd
df = pd.read_csv("fakeReviewDataLemmatized.csv")

6:

VECTORIZATION

In [14]:
from gensim.models import Word2Vec

# Train a Word2Vec model on the tokens
word2vec_model = Word2Vec(sentences=df['tokens'], vector_size=100, window=5, min_count=1, workers=4)

# Example: Get the embedding for a word
print(word2vec_model.wv['love'])  # Replace 'love' with any word in your corpus

# Calculate sentence embeddings (average of word vectors)
df['sentence_embedding'] = df['tokens'].apply(
    lambda tokens: sum(word2vec_model.wv[token] for token in tokens if token in word2vec_model.wv) / len(tokens)
)

# Verify the embeddings
print(df['sentence_embedding'].head())


[-1.9379928  -2.2582974   2.0822835  -0.14541158  1.766413   -0.5553301
 -2.3091087  -0.46849772 -0.35740802  1.4148506   0.11011387  0.10354994
 -0.2593716  -0.55676     0.25424042  0.7575684  -0.7773697  -0.7532801
  1.7922465  -0.82747966  1.5918834   1.9882152   1.4189023   1.930057
 -1.2273922  -2.2690227   2.6369216  -1.5885016  -1.7315971  -1.9388423
  0.8566928  -1.6960273   1.8993409   1.753758    2.9653609  -1.7323052
 -0.72560734 -1.4164388  -1.8512759  -0.48776168  3.9412568   3.4486923
  0.14409673 -0.6739967  -0.83567905 -0.41162592 -1.3563464  -0.3039591
 -0.08753868 -3.4055538   1.5397207  -2.2270803   0.81422985 -0.8166975
 -1.7349569  -0.39080548 -0.02773979  1.0049574  -1.8453878  -3.6597562
 -0.46632826 -1.3162625  -0.71918184  0.06219748  1.6589711   1.1551015
 -0.76251054 -0.75425065  1.8021358  -0.02483344  1.7671657   1.2352922
  0.06451735  0.16145071 -0.1312562  -1.5268155   1.9326936   1.6353112
  0.54105705  1.9310467  -2.007759    1.7014151  -1.4828423  -0.

In [16]:
# Save the Word2Vec model
word2vec_model.save("word2vec_model.model")
